In [4]:
import numpy as np
import xarray as xr
from collections import OrderedDict

In [5]:
def ds_fix_dims(ds):
    '''
    Fix time dimensions to be easier to call, put time in hours and put x and y in km
    '''
    if "time_fine" not in ds.variables:
        ds = ds.rename({str(u'time_series_4500_1800.0'): 'time_coarse', str(u'time_series_75_60.0'): 'time_fine', str(u'time_series_75_1800.0'): 'time_mid'})
    ds['time_coarse']=ds.time_coarse/3600
    ds['time_mid']=ds.time_mid/3600
    ds['time_fine']=ds.time_fine/3600
    ds['x'] = ds.x.astype(float)/(35)
    ds['y'] = ds.y.astype(float)/(35)
    return ds

def surf_pre_calc(ds, when):
    '''
    surface precip domain mean in mm and mean over the simulation time
    '''
    if when == "beginning":
        surface_precip_mean = ds.surface_precip_mean[fine_spin:]*1000
        surface_precip_mean = surface_precip_mean.mean()/(ds.time_fine[-1]-ds.time_fine[fine_spin])
    elif when == "end":
        surface_precip_mean = ds.surface_precip_mean[-fine_ave:]*1000
        surface_precip_mean = surface_precip_mean.mean()/(ds.time_fine[-1]-ds.time_fine[fine_ave])
    return surface_precip_mean

def rwp_calc(ds, when):
    '''
    rwp domain mean in g m^(-2) and mean over the simulation time
    '''
    if when == "beginning":
        rwp_mean = ds.RWP_mean[fine_spin:]*1000
        rwp_mean = rwp_mean.mean()
    elif when == "end":
        rwp_mean = ds.RWP_mean[-fine_ave:]*1000
        rwp_end_mean = rwp_mean.mean()
    return rwp_mean

def rain(ds):
    rwp_mean = rwp_calc(ds, "beginning")
    rwp_end_mean = rwp_calc(ds, "end")
    sp_mean = surf_pre_calc(ds, "beginning")
    sp_end_mean = surf_pre_calc(ds, "end")
    return [rwp_mean.item(), rwp_end_mean.item(), sp_mean.item(), sp_end_mean.item()]

def tke_calc(ds): 
    tke_mean = ds.reske_mean + ds.subke_mean
    time_mean = ds.reske_mean[ds.reske_mean.dims[0]].values
    return tke_mean, time_mean

def lwp_cloud_calc(lmmr, lwp):   
    cloudy_lwp = []
    t = []
    lwp_masked_arrs=[]
    for m in range(len(lmmr)):
        col_mask = layer_cloud_mask(lmmr, m)
        arr_mask = col_mask.values
        lwp_masked = lwp[m].where(arr_mask==1)
        lwp_masked_arrs.append(lwp_masked*1000)
        cloud_lwp_mean = lwp_masked.mean(axis=(0,1))
        cloudy_lwp.append(cloud_lwp_mean.item()*1000)
        t.append(lmmr[m][lmmr.dims[0]].item())

    lwp_masked = lwp_masked*1000
    return cloudy_lwp, t, lwp_masked, lwp_masked_arrs

def lwp_cloud(ds):
    if include_spinup == True:
        lwp = ds.lwp
        lmmr = ds.q_cloud_liquid_mass
        time_arr = ds.time_coarse
    else:
        lwp = ds.lwp[coarse_spin:]
        lmmr = ds.q_cloud_liquid_mass[coarse_spin:]
        time_arr = ds.time_coarse[coarse_spin:]
    
    cloudy_lwp, times, lwp_masked_last, lwp_masked_arrs = lwp_cloud_calc(lmmr, lwp)
    lwp_cloud_mean = np.mean(cloudy_lwp[-coarse_ave:])
    lwp_cloud_teme = (np.mean(cloudy_lwp[-coarse_ave:])-np.mean(cloudy_lwp[:coarse_ave]))/(time_arr[-1] - time_arr[0])
    tendency = calc_tendency(cloudy_lwp, times)
    return [lwp_cloud_mean, lwp_cloud_teme],cloudy_lwp, tendency,times, lwp_masked_last, lwp_masked_arrs

def calc_tendency(dataarray, *times):
    tendency=[]
    if times:
        tseries = times[0]
    else:
        tseries = dataarray[dataarray.dims[0]].values
        dataarray = dataarray.values
    for t_ind in range(1, len(dataarray), 1):
        c_step = dataarray[t_ind]
        p_step = dataarray[t_ind - 1]
        dx = (c_step) - (p_step)
        t = tseries[t_ind] - tseries[t_ind-1]
        if t != 0:
            tendency.append(dx/t)
        else:
            tendency.append(dx/0.01)
    return tendency

def column_cloud_fraction(lmmr):
    cloud_frac=[]
    t =[]
    for m in range(len(lmmr)):
        col_mask = layer_cloud_mask(lmmr, m)
        total = col_mask.sum(axis=(0,1))
        f = total.item()/(250*250)
        cloud_frac.append(f)
        t.append(lmmr[m][lmmr.dims[0]].item())
    return cloud_frac, t

def clfrac(ds):
    if include_spinup == True:
        lmmr = ds.q_cloud_liquid_mass
        time_arr = ds.time_coarse
    else:
        lmmr = ds.q_cloud_liquid_mass[coarse_spin:]
        time_arr = ds.time_coarse[coarse_spin:]

    cloud_frac, times = column_cloud_fraction(lmmr)
    cloud_frac_mean = np.mean(cloud_frac[-coarse_ave:])
    cloud_frac_teme = (np.mean(cloud_frac[-coarse_ave:]) - np.mean(cloud_frac[:coarse_ave]))/(time_arr[-1] - time_arr[0])
    tendency = calc_tendency(cloud_frac, times)
    return [cloud_frac_mean, cloud_frac_teme], cloud_frac, tendency, times

def layer_cloud_mask(dataarray, time):
    '''
    Applies mask to each timestep and sums
    '''
    for n in range(110):
        layer = dataarray[time,:,:,n]
        dataarray[time,:,:,n] = layer.where(layer.values<1e-5,1).where(layer.values>1e-5,0)
    col_sum = dataarray[time].sum(axis=2,skipna=True)
    col_mask = col_sum.where(col_sum.values<1,1)
    return col_mask

In [6]:
### Load design and set paths to data
nd = "low_nd"  # "low_nd" or "high_nd"
ppe_path   = "/gws/nopw/j04/carisma/eers/dycoms/{}/PPE/ppe".format(nd)
val_path   = "/gws/nopw/j04/carisma/eers/dycoms/{}/VAL/val".format(nd)
extra_path = "/gws/nopw/j04/carisma/eers/dycoms/{}/EXTRA/extra".format(nd)
base_path  = "/gws/nopw/j04/carisma/eers/dycoms/{}/BASE/base/base.nc".format(nd)
design       = np.loadtxt("designs/EmulatorInputsDesign2D.csv",delimiter=",", skiprows=1)
validation   = np.loadtxt("designs/ValidationInputsDesign2D.csv", delimiter=",", skiprows=1)
extra_design = np.loadtxt("designs/extra_points.csv",delimiter=",")
base = np.array([[-7.5, 8.5]])

ppe_no   = 20
val_no   = 8
extra_no = 6
base_no  = 1

In [9]:
od = OrderedDict()
od["ppe"]   = [ppe_no, ppe_path, design]
od["val"]   = [val_no, val_path, validation]
od["extra"] = [extra_no, extra_path, extra_design]
od["base"]  = [base_no, base_path, base]

### Initialise np arrays ###
mean = np.empty((ppe_no+val_no+extra_no+base_no, 3))
teme = np.empty((ppe_no+val_no+extra_no+base_no, 3))

arrays = [mean, teme]

rwp     = np.empty((ppe_no+val_no+extra_no+base_no, 3))
rwp_end = np.empty((ppe_no+val_no+extra_no+base_no, 3))
surface_precip     = np.empty((ppe_no+val_no+extra_no+base_no, 3))
surface_precip_end = np.empty((ppe_no+val_no+extra_no+base_no, 3))

rain_arrays = [rwp, rwp_end, surface_precip, surface_precip_end]

In [10]:
### Options ###
include_spinup = False # See fine_spin and coarse_spin for setup

coarse_spin = 2   # 1 hour = 1, 2 hours = 2
fine_spin   = 111 # 1 hour = 55, 2 hours = 111
coarse_ave  = 3
fine_ave    = 111  

### Loop through PPE datasets and calculate the in-cloud LWP, cloud fraction or rain mass-mixing ratio ###
calc    = 'lwp_cloud'      # options: 'lwp_cloud', 'cloud_frac', 'rain'
type_nd = "lwp_low"  # [lwp/cf/rain]_[low/high] but do lwp_total_low/high if doing lwp_total.
i=0

type_timeseries=[]

for key in od:
    for j in range(od[key][0]):
        k = j+1
        nc = od[key][1] + str(k) + "/" + key + str(k) + ".nc"
        if key=='base':
            nc = base_path 
        ds = xr.open_dataset(nc)
        ds = ds_fix_dims(ds)
            
        if calc == 'lwp_cloud':
            output_array, timeseries, tendency, times, lwp_masked_last, lwp_masked_arrs = lwp_cloud(ds)
            time_hrs = times

            ### uncomment to save top-down scenes
            # for step, masked_arr in enumerate(lwp_masked_arrs):
            #     np.savetxt(f"{key}_scenes/scene_{step}.csv",masked_arr.values, delimiter=',')

        elif calc == 'cloud_frac':
    	    output_array, timeseries, tendency, times = clfrac(ds)
    	    time_hrs = times
        elif calc == 'rain':
    	    output_array = rain(ds)
        else:
            print('Select calc')
            break

        if calc in ['cloud_frac','lwp_cloud']:
    	    for b, array in enumerate(arrays):
                array[i, 0] = od[key][2][j][1]   # save theta input value
                array[i, 1] = od[key][2][j][0]   # save qt input value
                array[i, 2] = output_array[b]    # save output value
        elif calc == 'rain':
    	    for b, array in enumerate(rain_arrays):
    	        array[i, 0] = od[key][2][j][1]   # save theta input value
    	        array[i, 1] = od[key][2][j][0]   # save qt input value
    	        array[i, 2] = output_array[b]    # save output value

        ### uncomment to save timeseries
        # if key=='ppe':
        #     type_timeseries.append(timeseries)
        # elif key=='base':
        #     base=timeseries
        # np.savetxt('./timeseries/{}/{}{}_timeseries.csv'.format(type_nd,key,str(k)),timeseries,delimiter=',')   # need type_nd instead of calc if not doing lwp_cloud
        # np.savetxt('./timeseries/{}/{}{}_times.csv'.format(type_nd,key,str(k)),time_hrs,delimiter=',')
        ds.close()
        i+=1

In [ ]:
if calc == 'rain':
    np.savetxt("dycoms_data_{0}_{1}_rwp.csv".format(nd, calc), rain_arrays[0], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_rwp_end.csv".format(nd, calc), rain_arrays[1], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_surfpre.csv".format(nd,calc), rain_arrays[2], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_surfpre_end.csv".format(nd,calc), rain_arrays[3], delimiter=",")
else:
    print("Saving..")
    np.savetxt("dycoms_data_{0}_{1}_mean.csv".format(nd,calc), arrays[0], delimiter=",")
    np.savetxt("dycoms_data_{0}_{1}_teme.csv".format(nd,calc), arrays[1], delimiter=",")

In [ ]:
### Do calculations for initial-condition ensembles
#calc='cloud_frac'
calc='lwp_cloud'

type_nd = "lwp_cloud_low"
### Variability ###
var_path = "/gws/nopw/j04/carisma/eers/dycoms/low_nd/INT_VAR/" #.format(nd)
points = ['ppe3','ppe9','ppe11','ppe14','ppe15','ppe17','ppe18','ppe19','ppe20']

### Create lists ###
e_mean=[]
e_teme=[]
i=0
for point in points:
    for i in range(1,6):# dropped to 4 extras + the 1 original run
        if i==(5):
            ds=xr.open_dataset(f"/gws/nopw/j04/carisma/eers/dycoms/low_nd/PPE/{point}/{point}.nc")
            print('ds_last = original')
        else:
            name = '{0}_{1}'.format(point, i)
            nc = '{0}{1}/{2}.nc'.format(var_path,point,name)
            print(nc)
            ds = xr.open_dataset(nc)

        ds = ds_fix_dims(ds)

        elif calc == 'lwp_cloud':
            output_array, timeseries, tendency, times, lwp_masked_last, lwp_diff = lwp_cloud(ds)
            time_hrs = times
            ### uncomment to save top-down scenes
            # np.savetxt("lwp_scenes_last/lnd_lwp_scene_last_%s%s.csv"%(point,i),lwp_masked_last.values, delimiter=',')
            e_mean.append(output_array[0])
            e_teme.append(output_array[1])
        elif calc == 'cloud_frac':
            output_array, timeseries, tendency, times = clfrac(ds)
            time_hrs = times
            e_mean.append(output_array[0])
            e_teme.append(output_array[1])
        else:
            print("Select calc")
            break

        # np.savetxt('./timeseries/{}/{}{}_timeseries.csv'.format(type_nd,point,str(i+1)), timeseries,delimiter=',')
        # np.savetxt('./timeseries/{}/{}{}_times.csv'.format(type_nd,point,str(i+1)),time_hrs,delimiter=',')

    #print(point + ' finished')    

elif calc=='cloud_frac':
    out_list = [e_mean, e_teme]
elif calc=='lwp_cloud':
    out_list = [e_mean,e_teme]
out_arr_T = np.array(out_list).T
np.savetxt('ensemble_{}_mean.csv'.format(calc), out_arr_T, delimiter=',')